## How to Use in Fiesta API

Once connected, use these endpoints:

```bash
# Check available models
curl http://localhost:8000/chat/models

# Send a chat message (uses tokens)
curl -X POST http://localhost:8000/chat/ \
  -H "user-id: {user_id}" \
  -H "Content-Type: application/json" \
  -d '{
    "prompt": "What is AI?",
    "model": "gemini-1.5-pro",
    "max_tokens": 500
  }'

# Check tokens remaining
curl http://localhost:8000/subscriptions/{user_id}

# Check API costs so far
curl http://localhost:8000/admin/costs/{user_id}
```

### Token Flow in Each Request:

1. **Request arrives**: User sends prompt + model choice
2. **Check subscription**: Verify user has active subscription + tokens remaining
3. **Check model access**: Verify user's tier allows that model
4. **Check rate limit**: Verify user hasn't exceeded requests/minute
5. **Estimate tokens**: Count tokens in prompt
6. **Call API**: Send to OpenAI/Claude/Gemini
7. **Get response**: Receive tokens_used from API
8. **Deduct tokens**: `tokens_remaining -= tokens_used`
9. **Track cost**: `monthly_api_cost_usd += cost`
10. **Return response**: Send to user

In [25]:
import pandas as pd

# Create summary table
summary_data = {
    "Provider": ["Gemini", "Anthropic", "OpenAI"],
    "Status": [
        "✓ Connected" if gemini_status["connected"] else "✗ Failed",
        "✓ Connected" if anthropic_status["connected"] else "✗ Failed",
        "✓ Connected" if openai_status["connected"] else "✗ Failed"
    ],
    "Error": [
        gemini_status["error"] or "None",
        anthropic_status["error"] or "None",
        openai_status["error"] or "None"
    ]
}

df = pd.DataFrame(summary_data)
print("\n" + "=" * 80)
print("CONNECTION STATUS SUMMARY")
print("=" * 80)
print(df.to_string(index=False))
print("=" * 80)

# Count working providers
working = sum([gemini_status["connected"], anthropic_status["connected"], openai_status["connected"]])
print(f"\n✓ {working}/3 providers are connected and working!")
print("\nYou can now use these providers in the Fiesta API:")


CONNECTION STATUS SUMMARY
 Provider      Status Error
   Gemini ✓ Connected  None
Anthropic ✓ Connected  None
   OpenAI ✓ Connected  None

✓ 3/3 providers are connected and working!

You can now use these providers in the Fiesta API:


## Display Connection Status Summary

In [24]:
openai_status = {"connected": False, "error": None, "response": None}

try:
    from openai import OpenAI
    
    openai_key = os.getenv("OPENAI_API_KEY")
    if not openai_key:
        raise Exception("OPENAI_API_KEY not set in .env")
    
    client = OpenAI(api_key=openai_key)
    
    # Test request
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": "Say 'Hello from Fiesta!' in 3 words"}
        ],
        max_tokens=100
    )
    
    openai_status["connected"] = True
    openai_status["response"] = response.choices[0].message.content
    
    print("✓ OpenAI API Connection: SUCCESS")
    print(f"Response: {openai_status['response']}")
    print(f"Model: {response.model}")
    print(f"Usage: {response.usage.prompt_tokens} input, {response.usage.completion_tokens} output")
    
except Exception as e:
    openai_status["error"] = str(e)
    print(f"✗ OpenAI API Connection: FAILED")
    print(f"Error: {openai_status['error']}")

✓ OpenAI API Connection: SUCCESS
Response: Hello from Fiesta!
Model: gpt-3.5-turbo-0125
Usage: 17 input, 4 output


## Test OpenAI API Connection

In [16]:
anthropic_status = {"connected": False, "error": None, "response": None}

try:
    from anthropic import Anthropic
    
    anthropic_key = os.getenv("ANTHROPIC_API_KEY")
    if not anthropic_key:
        raise Exception("ANTHROPIC_API_KEY not set in .env")
    
    client = Anthropic(api_key=anthropic_key)
    
    # Test request
    message = client.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=100,
        messages=[
            {"role": "user", "content": "Say 'Hello from Fiesta!' in 3 words"}
        ]
    )
    
    anthropic_status["connected"] = True
    anthropic_status["response"] = message.content[0].text if message.content else "No response"
    
    print("✓ Anthropic API Connection: SUCCESS")
    print(f"Response: {anthropic_status['response']}")
    print(f"Model: {message.model}")
    print(f"Usage: {message.usage.input_tokens} input, {message.usage.output_tokens} output")
    
except Exception as e:
    anthropic_status["error"] = str(e)
    print(f"✗ Anthropic API Connection: FAILED")
    print(f"Error: {anthropic_status['error']}")

✓ Anthropic API Connection: SUCCESS
Response: Fiesta says hello!
Model: claude-3-haiku-20240307
Usage: 21 input, 9 output


## Test Anthropic API Connection

In [15]:
gemini_status = {"connected": False, "error": None, "response": None, "model_used": None}

try:
    import google.generativeai as genai
    
    gemini_key = os.getenv("GEMINI_API_KEY")
    if not gemini_key:
        raise Exception("GEMINI_API_KEY not set in .env")
    
    genai.configure(api_key=gemini_key)
    
    # Try latest model first: gemini-2.5-pro
    model_to_try = "gemini-2.5-pro"
    
    try:
        model = genai.GenerativeModel(model_to_try)
        response = model.generate_content("Say 'Hello from Fiesta!' in 3 words")
        gemini_status["connected"] = True
        gemini_status["response"] = response.text if hasattr(response, 'text') else str(response)
        gemini_status["model_used"] = model_to_try
    except Exception as e1:
        # Fallback: try gemini-2.5-flash
        print(f"Model {model_to_try} failed: {str(e1)}")
        model_to_try = "gemini-2.5-flash"
        print(f"Trying fallback model: {model_to_try}")
        model = genai.GenerativeModel(model_to_try)
        response = model.generate_content("Say 'Hello from Fiesta!' in 3 words")
        gemini_status["connected"] = True
        gemini_status["response"] = response.text if hasattr(response, 'text') else str(response)
        gemini_status["model_used"] = model_to_try
    
    print("✓ Gemini API Connection: SUCCESS")
    print(f"Model used: {gemini_status['model_used']}")
    print(f"Response: {gemini_status['response']}")
    
except Exception as e:
    gemini_status["error"] = str(e)
    print(f"✗ Gemini API Connection: FAILED")
    print(f"Error: {gemini_status['error']}")


✓ Gemini API Connection: SUCCESS
Model used: gemini-2.5-pro
Response: Hello from Fiesta!


## Test Gemini API Connection

## List Available Gemini Models

In [14]:
try:
    import google.generativeai as genai
    
    gemini_key = os.getenv("GEMINI_API_KEY")
    if not gemini_key:
        print("✗ GEMINI_API_KEY not set in .env")
    else:
        genai.configure(api_key=gemini_key)
        
        print("Available Gemini models that support generateContent:")
        print("=" * 60)
        available_models = []
        for m in genai.list_models():
            if 'generateContent' in m.supported_generation_methods:
                available_models.append(m.name)
                print(f"  {m.name}")
        
        if not available_models:
            print("  (No models found - check API key permissions)")
        print("=" * 60)
        
except Exception as e:
    print(f"✗ Error listing Gemini models: {str(e)}")


Available Gemini models that support generateContent:
  models/gemini-2.5-flash
  models/gemini-2.5-pro
  models/gemini-2.0-flash-exp
  models/gemini-2.0-flash
  models/gemini-2.0-flash-001
  models/gemini-2.0-flash-lite-001
  models/gemini-2.0-flash-lite
  models/gemini-2.0-flash-lite-preview-02-05
  models/gemini-2.0-flash-lite-preview
  models/gemini-2.0-pro-exp
  models/gemini-2.0-pro-exp-02-05
  models/gemini-exp-1206
  models/gemini-2.5-flash-preview-tts
  models/gemini-2.5-pro-preview-tts
  models/gemma-3-1b-it
  models/gemma-3-4b-it
  models/gemma-3-12b-it
  models/gemma-3-27b-it
  models/gemma-3n-e4b-it
  models/gemma-3n-e2b-it
  models/gemini-flash-latest
  models/gemini-flash-lite-latest
  models/gemini-pro-latest
  models/gemini-2.5-flash-lite
  models/gemini-2.5-flash-image-preview
  models/gemini-2.5-flash-image
  models/gemini-2.5-flash-preview-09-2025
  models/gemini-2.5-flash-lite-preview-09-2025
  models/gemini-3-pro-preview
  models/gemini-3-pro-image-preview
  model

In [13]:
api_keys = {
    "GEMINI": os.getenv("GEMINI_API_KEY", "NOT SET"),
    "ANTHROPIC": os.getenv("ANTHROPIC_API_KEY", "NOT SET"),
    "OPENAI": os.getenv("OPENAI_API_KEY", "NOT SET")
}

print("API Keys Status:")
print("=" * 50)
for provider, key in api_keys.items():
    status = "✓ SET" if key != "NOT SET" else "✗ NOT SET"
    key_preview = f"{key[:10]}..." if len(key) > 10 else key
    print(f"{provider:12} {status:15} {key_preview}")

API Keys Status:
GEMINI       ✓ SET           AIzaSyC0NF...
ANTHROPIC    ✓ SET           sk-ant-api...
OPENAI       ✓ SET           sk-proj-Kf...


## Set Up API Keys

In [12]:
import os
import sys
from pathlib import Path

# Add project to path
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Load .env
from dotenv import load_dotenv
load_dotenv()

print("✓ Libraries imported")

✓ Libraries imported


## Import Required Libraries

# LLM Provider Connection Test

This notebook tests connectivity to all three LLM providers (Gemini, Anthropic, OpenAI) and explains how token consumption works in the Fiesta API.

## How Token Usage Works

1. **User creates account** → Gets subscription (free = 1000 tokens/month)
2. **User calls `/chat/` endpoint** → Fiesta checks if they have tokens
3. **LLM API is called** → OpenAI/Claude/Gemini processes request
4. **Response received** → Tokens used = prompt tokens + completion tokens
5. **Tokens deducted** → `tokens_remaining -= tokens_used`
6. **Cost tracked** → `monthly_api_cost_usd` increases (for billing)

Each provider charges differently:
- **OpenAI**: GPT-3.5 costs ~$0.002/request, GPT-4 ~$0.015/request
- **Anthropic**: Claude 3 Haiku ~$0.001/request, Opus ~$0.075/request  
- **Gemini**: Free tier, then $0.00025 per 1K input tokens